单件模式 Singleton Pattern
===

> 确保一个类只有一个实例，并提供一个全局访问点
>
> 下述的例子中没有考虑线程安全，请按照双重检查加锁的原则进行改写

In [1]:
class Singleton:

    """通过重载 new 方法
        保存类变量 _instance 的方式来实现单例
    """

    def __new__(cls, *args, **kw):
        if not hasattr(cls, '_instance'):
            cls._instance = super().__new__(cls, *args, **kw)

        return cls._instance

    
a = Singleton()
b = Singleton()
a is b

True

In [2]:
class Singleton:

    """通过共享属性来模拟单例
        其实不是严格的单例，但是实例间共享属性
    """

    _state = {}

    def __new__(cls, *args, **kw):
        ob = super().__new__(cls, *args, **kw)
        ob.__dict__ = cls._state

        return ob


    
a = Singleton()
b = Singleton()
a.a = '123123'
a.a is b.a

True

In [3]:
class MetaSigleton(type):
    
    """通过元类来实现单例
        元类是指导类生成的，生成一个包含类变量的类，
        并定义类调用时的 __call__ 方法
    """

    def __init__(cls, name, bases, dict):
        super().__init__(name, bases, dict)
        cls._instance = None  # 初始化类

    def __call__(cls, *args, **kw):
        """类实例化的时，先于 __new__
        若已实例化过，直接返回实例
        """
        if cls._instance is None:
            cls._instance = super().__call__(*args, **kw)

        return cls._instance

    
class Singleton(metaclass=MetaSigleton):
    pass


a = Singleton()
b = Singleton()
a is b

True

In [4]:
from functools import wraps


def singleton(cls):
    instances = {}

    @wraps(cls)
    def wrapper(*args, **kw):
        """其实类似于用元类来复写 __call__
            不过是用类修饰器来实现
        """
        if cls not in instances:
            instances[cls] = cls(*args, **kw)

        return instances[cls]

    return wrapper


@singleton
class Singleton:
    pass


a = Singleton()
b = Singleton()
a is b

True